In [128]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
tf.__version__

'2.4.1'

In [129]:
traindata = pd.read_csv('training_set_features.csv')
traindata.tail()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg
26706,26706,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,mlyzmhmf,"MSA, Principle City",1.0,0.0,NaN,NaN


In [130]:
from sklearn.preprocessing import LabelEncoder

df = traindata

def transform(df):
    def sextobinary(x):
        if(x == 'Male'): return 1;
        else: return 0;

    def marriedbinary(x):
        if(x == 'Married'): return 1;
        else: return 0;


    def rent(x):
        if(x == 'Own'): return 1;
        else: return 0;

    df['sex'] = df['sex'].apply(lambda x: sextobinary(x))
    df['marital_status'] = df['marital_status'].apply(lambda x: marriedbinary(x))
    df['rent_or_own'] = df['rent_or_own'].apply(lambda x: rent(x))
    
    
    le = LabelEncoder()
    for i in df.columns:
        if(df[i].dtype == 'object'):
            df[i] = le.fit_transform(df[i])
            
    df.fillna(method ='ffill')
    return df

In [131]:
df = transform(traindata)

In [132]:
testdata = pd.read_csv('training_set_labels.csv')
testdata.shape

(26707, 3)

In [133]:
h1_vac = testdata['h1n1_vaccine']
seasonal_vac = testdata['seasonal_vaccine']

In [134]:
from tensorflow.keras import Input, Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling1D, Embedding, Dropout, BatchNormalization

act = tf.nn.leaky_relu
model = Sequential([
    Dense(128, activation = act),
    BatchNormalization(),
    Dense(64, activation = act),
    Dense(1, activation='sigmoid' )
])
model.compile(optimizer = 'sgd',loss = 'mse', metrics = [tf.keras.losses.Huber()])
model.build(df.shape)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (26707, 128)              4736      
_________________________________________________________________
batch_normalization_7 (Batch (26707, 128)              512       
_________________________________________________________________
dense_22 (Dense)             (26707, 64)               8256      
_________________________________________________________________
dense_23 (Dense)             (26707, 1)                65        
Total params: 13,569
Trainable params: 13,313
Non-trainable params: 256
_________________________________________________________________


In [135]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopper = EarlyStopping(monitor = 'huber_loss',patience =2)
cbs = [early_stopper]
def train_model(target, model = model, data = df):
    history = model.fit(data, target,validation_split = 0.15,  epochs = 20, batch_size = 64,shuffle = True,callbacks = cbs, verbose = 1)

In [136]:
testdata = pd.read_csv('test_set_features.csv')
testdata.shape

(26708, 36)

In [137]:
testdata = transform(testdata)
testdata.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1,0,0,0,7,0,1.0,0.0,1,7
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,0,0,0,1,2,3.0,0.0,1,20
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1,1,1,0,5,2,1.0,0.0,10,12
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,1,5,0,1.0,0.0,21,23
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0,0,1,0,6,2,0.0,1.0,4,10


In [138]:
sub = pd.read_csv('submission_format.csv')

In [139]:
train_model(h1_vac)
sub['h1n1_vaccine'] = model.predict(testdata)

Epoch 1/20
355/355 [==============================] - 1s 2ms/step - loss: nan - huber_loss: nan - val_loss: nan - val_huber_loss: nan
Epoch 2/20
355/355 [==============================] - 1s 2ms/step - loss: nan - huber_loss: nan - val_loss: nan - val_huber_loss: nan


In [140]:
sub.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,NaN,0.7
1,26708,NaN,0.7
2,26709,NaN,0.7
3,26710,NaN,0.7
4,26711,NaN,0.7


In [141]:
train_model(seasonal_vac)
sub['seasonal_vaccine'] = model.predict(testdata)

Epoch 1/20
355/355 [==============================] - 1s 2ms/step - loss: nan - huber_loss: nan - val_loss: nan - val_huber_loss: nan
Epoch 2/20
355/355 [==============================] - 1s 2ms/step - loss: nan - huber_loss: nan - val_loss: nan - val_huber_loss: nan


In [94]:
sub.to_csv('submission.csv')